In [1]:
import tensorflow as tf
import gc
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], True)


from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)



INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2070 SUPER, compute capability 7.5
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
Compute dtype: float16
Variable dtype: float32


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "D:/extended dataset/Train"

CATEGORIES = ["yes", "no"]

training_data = []

IMG_SIZE = 50

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                print(e)
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))


100%|██████████| 1050/1050 [00:01<00:00, 886.62it/s]

2100


In [3]:
import random

random.shuffle(training_data)

In [4]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X)
X= X/255.0
y = np.array(y)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [6]:
len(y_test)

210

In [7]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [8]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]
    
    data = list(zip(image_list, label_list))

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [9]:
clients = create_clients(X_train, y_train, num_clients=5, initial='client')

In [10]:

def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [11]:
#process and batch the training data for each client
clients_batched = dict()
co = 0
for (client_name, data) in clients.items():
    co+=1
    clients_batched[client_name] = batch_data(data)
    
   
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [12]:
clients_batched

{'client_1': <BatchDataset shapes: ((None, 50, 50, 3), (None, 2)), types: (tf.float64, tf.float32)>,
 'client_2': <BatchDataset shapes: ((None, 50, 50, 3), (None, 2)), types: (tf.float64, tf.float32)>,
 'client_3': <BatchDataset shapes: ((None, 50, 50, 3), (None, 2)), types: (tf.float64, tf.float32)>,
 'client_4': <BatchDataset shapes: ((None, 50, 50, 3), (None, 2)), types: (tf.float64, tf.float32)>,
 'client_5': <BatchDataset shapes: ((None, 50, 50, 3), (None, 2)), types: (tf.float64, tf.float32)>}

In [13]:
del(X_train)
del(y_train)
del(training_data)

In [14]:
#test_batched

In [15]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop


class SimpleModel:
    def build(self):
        model = VGG16(include_top=False,input_shape=(50, 50, 3), weights='imagenet')
        transfer_layer = model.get_layer('block5_pool')
        conv_model = Model(inputs=model.input, outputs=transfer_layer.output)
        # Start a new Keras Sequential model.
        new_model = Sequential()

        # Add the convolutional part of the VGG16 model from above.
        new_model.add(conv_model)

        # Flatten the output of the VGG16 model because it is from a
        # convolutional layer.
        new_model.add(Flatten())

        # Add a dense (aka. fully-connected) layer.
        # This is for combining features that the VGG16 model has
        # recognized in the image.

        new_model.add(Dense(512, activation='relu'))


        # Add the final layer for the actual classification.
        new_model.add(Dense(2, activation='softmax'))
    
        return new_model

In [16]:
from tensorflow.keras.optimizers import SGD
optimizer = Adam(lr=.00001)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']

D:\Anaconda\envs\tf_gpu_final\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [17]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    print(global_count)
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [18]:

#X = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
#l_X =  tf.data.Dataset.from_tensor_slices((list(X)[0:52], list(y_train)[0:52]))
#final_lx = l_X.batch(16)

In [19]:
#smlp_global = SimpleModel()
#local_model = smlp_global.build()
#local_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        
#set local model weight to the weight of the global model
#local_model.set_weights(global_weights)
        
#fit local model with client's data
#global_weights = global_model.get_weights()
#local_model.set_weights(global_weights)

#local_model.fit(clients_batched['client_20'], epochs=15, verbose=1)

"""    

scaled_local_weight_list = list()
scaling_factor = weight_scalling_factor(clients_batched, clients_batched['client_20'])
scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
scaled_local_weight_list.append(scaled_weights)
global_model.set_weights(average_weights)
average_weights = sum_scaled_weights(scaled_local_weight_list)
del local_model
K.clear_session()
"""
#for element in final_lx:
#  print(element)
#final_lx

"    \n\nscaled_local_weight_list = list()\nscaling_factor = weight_scalling_factor(clients_batched, clients_batched['client_20'])\nscaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)\nscaled_local_weight_list.append(scaled_weights)\nglobal_model.set_weights(average_weights)\naverage_weights = sum_scaled_weights(scaled_local_weight_list)\ndel local_model\nK.clear_session()\n"

In [20]:
smlp_global = SimpleModel()
global_model = smlp_global.build()
from tensorflow.keras import backend as K
from sklearn.metrics import accuracy_score

local_model = smlp_global.build()
local_model.compile(optimizer= 'adam', loss=loss, metrics=metrics)

In [23]:
import multiprocessing
comms_round = 20
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    
    
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    #random.shuffle(client_names)
    
    #loop through each client and create new local model
    count = 0
    for client in client_names:
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=1)
        test_model(X_test, y_test, local_model, comm_round)
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        
        #K.clear_session()
        #gc.collect()
        #del local_model
    
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

12/12 [==============================] - 1s 33ms/step - loss: 0.4912 - categorical_accuracy: 0.7487
comm_round: 0 | global_acc: 73.810% | global_loss: 0.5679758191108704
1920
12/12 [==============================] - 0s 32ms/step - loss: 0.5010 - categorical_accuracy: 0.7751
comm_round: 0 | global_acc: 77.143% | global_loss: 0.5683660507202148
1920
12/12 [==============================] - 0s 32ms/step - loss: 0.5529 - categorical_accuracy: 0.7328
comm_round: 0 | global_acc: 76.667% | global_loss: 0.5753054022789001
1920
12/12 [==============================] - 0s 31ms/step - loss: 0.5412 - categorical_accuracy: 0.7222
comm_round: 0 | global_acc: 74.762% | global_loss: 0.5991712808609009
1920
12/12 [==============================] - 0s 31ms/step - loss: 0.4854 - categorical_accuracy: 0.7804
comm_round: 0 | global_acc: 65.714% | global_loss: 0.6011168956756592
1920
comm_round: 0 | global_acc: 75.714% | global_loss: 0.5629990696907043
12/12 [==============================] - 0s 32ms/step -

In [22]:
predict_x=global_model.predict(X_train) 
classes_x=np.argmax(predict_x,axis=1)

NameError: name 'X_train' is not defined

In [ ]:
for i in classes_x:
    print(i)